In [ ]:
from datetime import datetime
from datetime import timedelta
import json 
import requests

URL = "https://api.wit.ai/message"
API_KEY = "6LLX376UX4FWFSX5KVNLSRPCXSECLGXE"
headers  = {
	"Authorization" : "Bearer " + API_KEY
}

class WitAPI(object):
	def sendRequest(self, question):
		date = datetime.now()
		date = date.strftime("%Y%m%d")
		resp = requests.get(URL + "?v=" + date + "&q=" + question, headers = headers)
		if resp.status_code != 200:
		    print('error: ' + str(resp.status_code))
		    return None
		else:
		    print('Success')
		    return resp.json()

#! pip install timezonefinder[numba]
from timezonefinder import TimezoneFinder
from datetime import datetime
import pytz

class TimeChatbot(object):
	def getTimezone(self, latitude, longitude):
		tf = TimezoneFinder()
		return tf.timezone_at(lng=longitude, lat=latitude)

	def getLocalTime(self, timeZone = None):
		if timeZone is None:
			dateTime = datetime.now()
		else:
			tz = pytz.timezone(timeZone)
			dateTime = datetime.now(tz)
		
		return dateTime

#! pip install ipregistry
from ipregistry import IpregistryClient
from geopy.geocoders import Nominatim
import geopy.distance

class Location(object):
	def getLocation(self, place = None):
		if place is None:
			client = IpregistryClient("fs9pbuwmnx5r2g")  
			ipInfo = client.lookup()
			#print(ipInfo)
			return(ipInfo.location.get("latitude"), ipInfo.location.get("longitude"))
		else:
			geolocator = Nominatim(user_agent="chatbot")
			location = geolocator.geocode(place)
			return (location.latitude, location.longitude)

	def distanceByLatLong(self, lat1, long1, lat2 = None, long2 = None):
		if lat2 is None:
			coord1 = (lat1, long1)
			coord2 = self.getLocation()
		else:
			coord1 = (lat1, long1)
			coord2 = (lat2, long2)

		return geopy.distance.distance(coord1, coord2).km

# 

chatbot = WitAPI()
question = 'nice to meet you'
chatbot.sendRequest(question)

loc = Location()
lat1 = loc.getLocation()[0]
long1 = loc.getLocation()[1]
loc.distanceByLatLong(lat1, long1, 50, 100)

time = TimeChatbot()
time.getTimezone(lat1, long1)
time.getLocalTime()

loc.getLocation('New York')

error: 500


(40.7127281, -74.0060152)

In [ ]:
# you should install the package firstly
#!pip install pyowm 
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
import json 
import requests

class GeoInfo(object):
  # you should go to this URL: https://home.openweathermap.org/api_keys
    # to create your own key if you want to use this API
  # more info about this API: https://openweathermap.org/api/one-call-api
  api_key = "817d0ac612d439a9fe94c38f889fab28"
  owm = OWM(api_key)
  mgr = owm.weather_manager()

  def __init__(self, lat=43.6532, long=-79.3832):
    self.lat = lat
    self.long = long
    self.observation = mgr.weather_at_coords(self.lat, self.long)
    self.w = self.observation.weather
  
  def get_temperature(self, lat=None, long=None, wtype='celsius'):
    if lat is not None and long is not None:
      self.lat = lat
      self.long = long
    self.observation = mgr.weather_at_coords(self.lat, self.long)
    self.w = self.observation.weather
    # print(w.temperature('celsius'))
    # w.to_dict()
    return w.temperature(wtype)
  '''
  def get_temperature(self, name='Toronto'):
    observation = mgr.weather_at_place(name)
    w = observation.weather
    print(w.temperature('celsius'))
    return w.to_dict()
  '''

  def get_weather(self):
    return self.w.detailed_status
  
  # limit, how many POIs you want to retrieve
  def get_point_of_interest(self, limit=5):
    URL = "https://discover.search.hereapi.com/v1/discover"
    latitude = self.lat
    longitude = self.long
    # get your key from developer.here.com
    poi_key = 'IIlHrKDojjhW90OB3KCEp6Xu0l8FX13NAdgFxMx4HbM' 
    query = 'tourist attractions'
    #print(latitude, longitude)
    PARAMS = {
          'apikey':poi_key,
          'q':query,
          'limit': limit,
          'at':'{},{}'.format(latitude,longitude)
          } 
    # sending request and getting the response as JSON
    r = requests.get(url = URL, params = PARAMS) 
    data = r.json()
    items = data['items']
    attractions = []
    # parse retrieved attractions data
    for it in items:
      att = 'distance: ' + str(it['distance']) + ', ' + it['address']['label']
      attractions.append(att)
    return attractions

# test
geoInfo = GeoInfo(22.2793278, 114.1628131)
# 43.6532, -79.3832 - Toronto / the default lat and long
# 51.5073219, -0.1276474 - London
# 40.7127281, -74.0060152 - New York
# 22.2793278, 114.1628131 - Hong Kong
# 39.906217, 116.3912757 - Beijing

temp = geoInfo.get_temperature()
wth = geoInfo.get_weather()
poi = geoInfo.get_point_of_interest(3)

print('temperature: ', temp)
print('weather: ', wth)
print('POI: ', poi)

temperature:  {'temp': 11.25, 'temp_max': 12.78, 'temp_min': 10.0, 'feels_like': 10.18, 'temp_kf': None}
weather:  broken clouds
POI:  ['distance: 110, Admiralty, De LiJie, Central, Hong Kong, Hong Kong SAR', 'distance: 227, 애버딘 항구, Xiang GangGong Yuan, Central, Hong Kong, Hong Kong SAR', 'distance: 227, Hong Kong SARHong KongCentralXiang GangGong YuanHong Kong And Macau Low Cost Tours']


In [ ]:
# reference: https://developer.here.com/blog/discovering-nearby-places-using-python-flask


URL = "https://discover.search.hereapi.com/v1/discover"
latitude = lat1
longitude = long1
poi_key = 'IIlHrKDojjhW90OB3KCEp6Xu0l8FX13NAdgFxMx4HbM' # Acquire from developer.here.com
query = 'attraction'
limit = 5

PARAMS = {
      'apikey':poi_key,
      'q':query,
      'limit': limit,
      'at':'{},{}'.format(latitude,longitude)
      } 
# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
data = r.json()
data
items = data['items']


In [ ]:
for it in items:
  print(it)
  print( 'distance: ', it['distance'], ', \t', it['address']['label'] )

{'title': 'Dalles Art Center', 'id': 'here:pds:place:840c21g6-f75615f5483342c5a1bfdaeec905561f', 'ontologyId': 'here:cm:ontology:tourist_attraction', 'resultType': 'place', 'address': {'label': 'Dalles Art Center, 220 E 4th St, The Dalles, OR 97058, United States', 'countryCode': 'USA', 'countryName': 'United States', 'stateCode': 'OR', 'state': 'Oregon', 'county': 'Wasco', 'city': 'The Dalles', 'street': 'E 4th St', 'postalCode': '97058', 'houseNumber': '220'}, 'position': {'lat': 45.59998, 'lng': -121.18361}, 'access': [{'lat': 45.60018, 'lng': -121.18346}], 'distance': 709, 'categories': [{'id': '300-3000-0023', 'name': 'Tourist Attraction', 'primary': True}, {'id': '300-3000-0024', 'name': 'Gallery'}, {'id': '300-3100-0000', 'name': 'Museum'}, {'id': '600-6900-0103', 'name': 'Gift, Antique & Art'}], 'references': [{'supplier': {'id': 'core'}, 'id': '1049286337'}, {'supplier': {'id': 'tripadvisor'}, 'id': '7852250'}, {'supplier': {'id': 'yelp'}, 'id': 'Q00O1zTk4d0-KDbl4wfYBQ'}], 'co

In [ ]:
w.detailed_status
w.wind()   
w.humidity   
w.temperature('celsius')
w.rain 
w.heat_index
w.clouds

mgr.one_call(lat1, long1)
ob = mgr.weather_at_coords(lat1, long1)
ob = mgr.weather_at_place('Beijing')
ob.location
ob.rec_time
w = ob.weather
print(w.temperature('celsius'))
w.to_dict()

'''

import requests, json 
api_key = "817d0ac612d439a9fe94c38f889fab28"
# 817d0ac612d439a9fe94c38f889fab28

#https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&appid={API key}
base_URL = 'https://api.openweathermap.org/data/2.5/onecall?'

req = base_URL + 'lat={'+str(lat1)+"}&lon={"+str(long1)+'}&appid{'+api_key+'}'

"""
base_url = "http://api.openweathermap.org/data/2.5/weather?"
city_name = input("Enter city name : ") 
complete_url = base_url + "appid=" + api_key + "&q=" + city_name 
response = requests.get(complete_url) 
x = response.json() 
"""
req
response = requests.get(req) 
x = response.json() 
x
'''